In [12]:
import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)

if (not logger.hasHandlers()):

    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.INFO)
    logger.addHandler(ch)

In [13]:
from pathlib import Path

year: int = 2006
tid: str = 'h09v05'
outDir: Path = '/explore/nobackup/people/rlgill/SystemTesting/modis-water'
modDir: Path = '/css/modis/Collection6.1/L2G'
startDay: int = 247
endDay: int = 247
sensors = set(['MOD'])

# This is for finding imports.  Point this to your local repository.
sys.path.append('/explore/nobackup/people/rlgill/innovation-lab-repositories/')


In [14]:
import numpy as np

from osgeo import gdal
from osgeo.osr import SpatialReference

from modis_water.model.BandReader import BandReader as br
from modis_water.model.Classifier import Classifier


# -----------------------------------------------------------------------------
# class DebugSimpleClassifier
# -----------------------------------------------------------------------------
class DebugSimpleClassifier(Classifier):

    CLASSIFIER_NAME = 'DebugSimple'

    # -------------------------------------------------------------------------
    # __init__
    # -------------------------------------------------------------------------
    def __init__(self, year, tile, outDir, modDir, startDay=1, endDay=365,
                 logger=None, sensors=set([br.MOD]), debug=False):

        super(DebugSimpleClassifier, self). \
            __init__(year, 
                     tile, 
                     str(outDir), 
                     str(modDir), 
                     startDay, 
                     endDay, 
                     logger,
                     sensors, 
                     debug,
                     [br.SOLZ, br.STATE, br.SR1, br.SR2, br.SR3, br.SR4,
                      br.SR5, br.SR6, br.SR7])

        self._noData: int = 250
        self._outDataType: int = gdal.GDT_Byte
        
    # -------------------------------------------------------------------------
    # createOutputImage
    # -------------------------------------------------------------------------
    def _createOutputImage(self, name, predictions):

        MODIS_SINUSOIDAL_6842 = SpatialReference('PROJCS["Sinusoidal",GEOGCS["GCS_Undefined",DATUM["Undefined",SPHEROID["User_Defined_Spheroid",6371007.181,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Sinusoidal"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]')  # noqa: E501

        driver = gdal.GetDriverByName('GTiff')

        ds = driver.Create(name, br.COLS, br.ROWS, 1, self._outDataType,
                           options=['COMPRESS=LZW'])

        ds.SetSpatialRef(MODIS_SINUSOIDAL_6842)
        ds.SetGeoTransform(self._bandReader.getXform())
        ds.SetProjection(self._bandReader.getProj())
        ds.GetRasterBand(1).SetNoDataValue(self._noData)
        ds.WriteRaster(0, 0, br.COLS, br.ROWS, predictions.tobytes())

        if self._debug and self._logger:
            self._logger.info('Writing image as ' + str(self._outDataType))

    # -------------------------------------------------------------------------
    # getClassifierName
    # -------------------------------------------------------------------------
    def getClassifierName(self):
        return DebugSimpleClassifier.CLASSIFIER_NAME

    # -------------------------------------------------------------------------
    # _runOneSensorOneDay
    # -------------------------------------------------------------------------
    def _runOneSensorOneDay(self, bandDict, outName):

        # Mark requested this currently unused variable to be defined.
        red = bandDict[br.SR1]
        
        # Name the arrays as named in water_change.c
        nir = bandDict[br.SR2]
        blue = bandDict[br.SR3]
        swir5 = bandDict[br.SR5]
        swir7 = bandDict[br.SR7]

        # ---
        # Add NDVI to bandDict.
        # NDVI is normally calculated with range -1,1. This multiplies
        # that range by 10,000 making it an integer-friendly range.
        # The NDVI conditions listed in water_change.c are multiplied
        # by 10,000 to match the range of the computed NDVI.
        # ---
        ndvi = self.computeNdvi(bandDict[br.SR1], bandDict[br.SR2])
        ndviBadCalculation = (bandDict[br.SR1] + bandDict[br.SR2]) == 0

        # ---
        # Define the rules as masks.
        # ---
        subcondition1 = (swir5 >= 453) & (blue < 675) & (nir > 1000)
        land1 = (swir5 < 1017) & (swir7 < 773) & subcondition1
        water1 = (swir5 < 1017) & (swir7 < 773) & ~subcondition1

        water2 = (swir5 >= 1017) & (nir < 1777) & (ndvi < 825) & \
                 (blue < 651)

        land2 = (swir5 >= 1017) & (nir < 1777) & (ndvi < 825) & \
            (blue >= 651)

        land3 = (swir5 >= 1017) & (nir < 1777) & (ndvi >= 825) & \
                (ndvi < 4125) & (nir >= 1329) & (swir7 < 1950)

        land4 = (swir5 >= 1017) & (nir < 1777) & (ndvi >= 825) & \
                (ndvi >= 4125)

        land5 = (swir5 >= 1017) & (nir >= 1777)

        waterConditions = water1 | water2
        landConditions = land1 | land2 | land3 | land4 | land5

        # Apply the model.
        predictions = np.full((br.COLS, br.ROWS), self._noData)
        
        predictions[land1] = 1
        predictions[land2] = 2
        predictions[land3] = 3
        predictions[land4] = 4
        predictions[land5] = 5
        predictions[water1] = 6
        predictions[water2] = 7
        
        predictions = np.where(ndviBadCalculation,
                               self._noData,
                               predictions)

        return predictions

In [15]:
classifier = DebugSimpleClassifier(year,
                                   tid,
                                   outDir,
                                   modDir,
                                   startDay=startDay, 
                                   endDay=endDay,
                                   logger=logger,
                                   sensors=sensors,
                                   debug=False)

classifier.run()

Reading MOD tile h09v05 for day 247
Reading MOD tile h09v05 for day 247
Creating /explore/nobackup/people/rlgill/SystemTesting/modis-water/2006-247-h09v05-MOD-Simple.tif
Creating /explore/nobackup/people/rlgill/SystemTesting/modis-water/2006-247-h09v05-MOD-Simple.tif
Generating mask
Generating mask
Classifiying
Classifiying
Masking
Masking
